# Introduction

I've found it helpful to have one notebook build up a cache of the data I want to analyze. I'm still not quite sure if anndata or loom is a better choice.

This notebook is to collect the 10x e10.5 forelimb runs.

In [1]:
import pandas
import scanpy
from pathlib import Path
import sys
from urllib import parse
import loompy
import numpy
import anndata
import scanpy

In [2]:
LRSC = str(Path('~/proj/long-rna-seq-condor').expanduser())
if LRSC not in sys.path:
    sys.path.append(LRSC)
from woldrnaseq.madqc import load_rsem_quantifications, load_genomic_quantifications, load_transcriptome_quantifications, replicate_scores
from woldrnaseq.models import load_library_tables, load_experiments, load_all_star_final, load_all_star_counts

# Load GTF

In [3]:
store = pandas.HDFStore(Path('~/proj/genome/GRCh38-V24-male/GRCh38-V24-male.h5').expanduser())
gtf = store[store.keys()[0]]
store.close()

In [27]:
gtf.shape

(2574639, 26)

In [4]:
gtf.columns

Index(['chromosome', 'source', 'type', 'start', 'stop', 'score', 'strand',
       'frame', 'gene_id', 'gene_type', 'gene_status', 'gene_name', 'level',
       'havana_gene', 'transcript_id', 'transcript_type', 'transcript_status',
       'transcript_name', 'tag', 'transcript_support_level',
       'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id',
       'ccdsid'],
      dtype='object')

In [5]:
gene_info = gtf[gtf['type'].isin(['gene', 'tRNA']) | (gtf['source'] == 'spikein')].set_index('gene_id')

# Load Transcript Map

In [6]:
grch38_v24_male_map = {}

for i, row in gtf[['gene_id', 'transcript_id']].iterrows():
    grch38_v24_male_map[row.transcript_id] = row.gene_id
    


In [49]:
evaluation_rsem_root = Path('~/proj/encode3-rna-evaluation/long-rna-seq-condor/').expanduser()
evaluation_kallisto_root = Path('~/proj/encode-202006-jamboree-detrout-rna-sc-pipeline/encode3-rna-evaluation-kallisto').expanduser()


# Load Population RSEM values

In [9]:
rsem_experiment = load_experiments([evaluation_rsem_root / 'experiment.tsv'])
rsem_libraries = load_library_tables([evaluation_rsem_root / 'library.tsv'])

In [10]:
star_stats = load_all_star_final(rsem_libraries)

In [11]:
star_stats.loc[:, ('', 'Number of input reads')]

ENCLB035ZZZ    25007733.0
ENCLB036ZZZ    25001532.0
ENCLB037ZZZ    25004651.0
ENCLB038ZZZ    25001913.0
ENCLB039ZZZ    25000903.0
ENCLB040ZZZ    25001076.0
ENCLB041ZZZ    24999822.0
ENCLB042ZZZ    25004947.0
ENCLB043ZZZ    25002616.0
ENCLB044ZZZ    25002815.0
ENCLB045ZZZ    25003023.0
ENCLB046ZZZ    25007451.0
Name: (, Number of input reads), dtype: float64

In [12]:
rsem_all = pandas.Series({'replicates': star_stats.index})

In [13]:
rsem_gene_count = load_genomic_quantifications(rsem_all, rsem_libraries, column='expected_count')
rsem_gene_tpm = load_genomic_quantifications(rsem_all, rsem_libraries, column='TPM')

quantifications None (61471, 12)
quantifications None (61471, 12)


In [14]:
rsem_transcript_count = load_transcriptome_quantifications(rsem_all, rsem_libraries, column='expected_count')
rsem_transcript_tpm = load_transcriptome_quantifications(rsem_all, rsem_libraries, column='TPM')

quantifications None (200094, 12)
quantifications None (200094, 12)


# Read STAR ReadsPerGene.tab

In [15]:
star_gene_count = load_all_star_counts(rsem_libraries, column='U').reindex(rsem_gene_count.index)

# Read kallisto results

In [16]:
def read_kallisto():
    libraries = [
        'ENCLB035ZZZ', 'ENCLB037ZZZ', 'ENCLB039ZZZ', 'ENCLB041ZZZ', 'ENCLB043ZZZ', 'ENCLB045ZZZ',
        'ENCLB036ZZZ', 'ENCLB038ZZZ', 'ENCLB040ZZZ', 'ENCLB042ZZZ', 'ENCLB044ZZZ', 'ENCLB046ZZZ',
    ]

    kallisto_transcript_tpm = {}
    kallisto_transcript_count = {}
    for library in libraries:
        abundance = pandas.read_csv(evaluation_kallisto_root / library / 'abundance.tsv', sep='\t', index_col=0, usecols=['target_id', 'est_counts', 'tpm'])
        kallisto_transcript_tpm[library] = abundance['tpm']
        kallisto_transcript_count[library] = abundance['est_counts']
        
    kallisto_transcript_tpm = pandas.DataFrame(kallisto_transcript_tpm)
    kallisto_transcript_count = pandas.DataFrame(kallisto_transcript_count)
    
    kallisto_transcript_tpm['gene_id'] = [grch38_v24_male_map[x] for x in kallisto_transcript_tpm.index]
    kallisto_transcript_count['gene_id'] = [grch38_v24_male_map[x] for x in kallisto_transcript_count.index]
    return kallisto_transcript_count, kallisto_transcript_tpm

kallisto_transcript_count, kallisto_transcript_tpm = read_kallisto()
kallisto_gene_count = kallisto_transcript_count.groupby('gene_id').sum()
kallisto_gene_tpm = kallisto_transcript_tpm.groupby('gene_id').sum()

# Save results

In [17]:
def build_loom(filename, matrix, quantification_name, gtf):
    gene_info = gtf[gtf['type'].isin(['gene', 'tRNA']) | (gtf['source'] == 'spikein')]
    transcript_info = gtf[(gtf['type'].isin(['transcript', 'tRNA'])) | (gtf['source'] == 'spikein')]
    
    if matrix.shape[0] == gene_info.shape[0]:
        # We have a gene matrix
        info = gene_info
        info = info.set_index('gene_id')
        feature_type = 'gene'
    elif matrix.shape[0] == transcript_info.shape[0]:
        info = transcript_info
        info = info.set_index('transcript_id')
        feature_type = 'transcript'
    else:
        raise ValueError('Unrecognized shape expected {} or {} got {}'.format(
            gene_info.shape[0], 
            transcript_info.shape[0],
            count.shape[0],
        ))
    gene_names = []
    gene_types = []
    for feature in matrix.index:
        gene_names.append(info.loc[feature, 'gene_name'])
        gene_types.append(info.loc[feature, 'gene_type'])

    row_attrs = {
        'id': numpy.asarray(matrix.index),
        'gene_name': numpy.asarray(gene_names),
        'gene_type': numpy.asarray(gene_types),
    }
    column_attrs = {
        'experiment': numpy.asarray(matrix.columns), 
    }
    file_attrs = {
        'quantification_name': quantification_name,
        'feature_type': feature_type,
    }
    loompy.create(str(filename), matrix.values, row_attrs=row_attrs, col_attrs=column_attrs, file_attrs=file_attrs)


In [54]:
def build_anndata(filename, matrix, quantification_name, gtf):
    gene_info = gtf[(gtf['type'] == 'gene') | (gtf['source'] == 'spikein')]
    transcript_info = gtf[(gtf['type'] == 'transcript') | (gtf['source'] == 'spikein')]
    
    gene_shape = 61471
    transcript_shape = 200094
    if matrix.shape[0] == gene_shape:
        # We have a gene matrixCell
        info = gene_info
        info = info.set_index('gene_id')
        feature_type = 'gene'
    elif matrix.shape[0] == transcript_shape:
        info = transcript_info
        info = info.set_index('transcript_id')
        feature_type = 'transcript'
    else:
        raise ValueError('Unrecognized shape expected {} or {} got {}'.format(
            gene_info.shape[0], 
            transcript_info.shape[0],
            matrix.shape[0],
        ))
    gene_names = []
    gene_types = []
    for feature in matrix.index:
        if feature in info.index:
            gene_names.append(info.loc[feature, 'gene_name'])
            gene_types.append(info.loc[feature, 'gene_type'])
        else:
            gene_names.append(feature)
            gene_types.append('tRNA')

    adata = anndata.AnnData(matrix.T)
    adata.var['gene_symbol'] = gene_names
    adata.var['gene_type'] = gene_types
    adata.uns['quantification_name'] = quantification_name
    adata.uns['feature_type'] = feature_type
    
    adata.write_h5ad(filename)

In [50]:
build_anndata(evaluation_kallisto_root / 'evaluation_rsem_gene_counts.h5ad', rsem_gene_count, 'counts', gtf)
build_anndata(evaluation_kallisto_root / 'evaluation_kallisto_gene_counts.h5ad', kallisto_gene_count, 'counts', gtf)

... storing 'gene_symbol' as categorical
... storing 'gene_type' as categorical
... storing 'gene_symbol' as categorical
... storing 'gene_type' as categorical


In [51]:
build_anndata(evaluation_kallisto_root / 'evaluation_rsem_gene_tpms.h5ad', rsem_gene_tpm, 'TPM', gtf)
build_anndata(evaluation_kallisto_root / 'evaluation_kallisto_gene_tpms.h5ad', kallisto_gene_tpm, 'TPM', gtf)

... storing 'gene_symbol' as categorical
... storing 'gene_type' as categorical
... storing 'gene_symbol' as categorical
... storing 'gene_type' as categorical


# Build Transcript Matrix

In [77]:
if 'gene_id' in rsem_transcript_count.columns:
    rsem_transcript_count = rsem_transcript_count.drop('gene_id', axis=1)
if 'gene_id' in kallisto_transcript_count.columns:
    kallisto_transcript_count = kallisto_transcript_count.drop('gene_id', axis=1)
    
assert 'gene_id' not in kallisto_gene_count.columns
    
build_anndata(evaluation_kallisto_root / 'evaluation_rsem_transcript_counts.h5ad', rsem_transcript_count, 'counts', gtf)
build_anndata(evaluation_kallisto_root / 'evaluation_kallisto_transcript_counts.h5ad', kallisto_transcript_count, 'counts', gtf)

... storing 'gene_symbol' as categorical
... storing 'gene_type' as categorical
... storing 'gene_symbol' as categorical
... storing 'gene_type' as categorical


In [78]:
if 'gene_id' in rsem_transcript_tpm.columns:
    rsem_transcript_tpm = rsem_transcript_tpm.drop('gene_id', axis=1)
if 'gene_id' in kallisto_transcript_tpm.columns:
    kallisto_transcript_tpm = kallisto_transcript_tpm.drop('gene_id', axis=1)

build_anndata(evaluation_kallisto_root / 'evaluation_rsem_transcript_tpm.h5ad', rsem_transcript_tpm, 'TPM', gtf)
build_anndata(evaluation_kallisto_root / 'evaluation_kallisto_transcript_tpm.h5ad', kallisto_transcript_tpm, 'TPM', gtf) 

... storing 'gene_symbol' as categorical
... storing 'gene_type' as categorical
... storing 'gene_symbol' as categorical
... storing 'gene_type' as categorical
